In [1]:
import os
# 下面注释掉的适合在非Pycharm 的Juypter Notebook上使用
# import sys
# path = os.path.join(os.path.dirname(os.getcwd()))
# sys.path.append(path) # 将所需要的根目录添加到路径
import torch
import numpy as np
import pandas as pd
from IPython.display import clear_output as clear
# 导入模型
from models.supervisedModels.GNNs import GeneralGNNs
# 导入小工具
from utils.common_utils import printlog
from trainTest.datasets.dataset_utils import get_fileName_weights, get_save_path, get_intra_dataloaders, get_print_info, get_pcc_knn_adj_from_dataloader
# 导入模型分类性能评价工具
from trainTest.train.intra_train_gnns import train_test_intra_model

In [2]:
### 1. 获取文件路径和文件名的设置
file_path = os.path.join(os.path.dirname(os.getcwd()) , 'preProcessing', 'trainData')
gait_or_motion = 'gait' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
subjects_list_global = list(['01', '02', '03', '04', '06', '31', '32', '33', '34', '36'])

### 2. 获取dataloaders的设置
total_exp_time = 5 
train_batch, valid_batch, test_batch = 32, 32, 32
data_list = ['sub_emg_sample', 'sub_angle_sample']
feature_list = ['sub_emg_features', 'sub_angle_features']

### 3. GeneralCNNs模型构建参数设置
node_amount = 15
## 3.1. edge_gen_params
#      1) max_esr_ratio: TRG图中保留的最大边比例 max_edge_number = math.floor(max_esr_ratio*node_amount); 
#      2) trg_edge_connect_mode: 'graph_fix' / 'node_fix' 则为max_edge_number; 'node_random'则为random.choice(list(range(1, max_edge_number)))
#      3) node_embedding: 节点嵌入，对'node_fix'和'node_random'，设为False较好；对'graph_fix'，可尝试设为True 
#      4) pcc_kgnn_gen_mode: 'batch_cnn_feature' / 'batch_train_data' / 'train_set'
#        'batch_cnn_feature'和'batch_raw_data'是每个batch都生成不同的动态图，计算开销会较大   
#        'train_set'是根据所有的训练集数据生成一张静态图，这种模式下，GCNs在处理KNN图和PCC图时，可以先用阈值激活，再赋予可训练的边权
#      5) kgnn_ratio: KNN图中K近邻的比例; 
#      6) pcc_act_thr / kgnn_act_thr / awmf_act_thr: 用于控制边连接是否激活的阈值。具体分析如下：
#         6.1) 当pcc_kgnn_gen_mode等于'batch_cnn_feature'或者'batch_train_data'时:
#              pcc_act_thr控制单个batch单个样本的PCC是否激活，设置为[0.5-0.9]
#         6.2) 当pcc_kgnn_gen_mode等于'train_set'时:
#              类似于一种投票机制，如果训练集中有不少于pcc_act_thr比例的样本所对应的PCC或不少于kgnn_act_thr比例的样本KNN图都有某个边连接，则激活. 设置为[0.1-0.9]
#         6.3) awmf_act_thr：一种投票激活机制，和可训练的graph_fusion_weight一起控制AWMF。假设有n个图融合，设置为>=1/n, 保证至少有一种图结构
#      7) AWMF_type: 多图融合的哪几种图。assert 'TRG' in self.AWMF_type and 4 >= len(self.AWMF_type) >= 2 / assert 4 >= len(self.AWMF_type) >= 2
#      8) AWMF之前，要对TRG，PCC和KNN寻优。例如，trg的max_esr_ratio，PCC的pcc_kgnn_act_thr，KNN的kgnn_ratio和edge_act_thr
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}
## 3.2. 其他params
network_branch = 1  #  1) network_branch = 1，只使用原始数据; 2)  network_branch = 2，同时使用原始数据和时域特征.
conv_type = 'Conv' # # one of [DNN, Conv, SKConv, ResNetV1, ResNetV2, MobileNetV1, MobileNetV2, MobileNetV3, ResNeSt, ShuffleNetV1, ShuffleNetV2]
edge_gen_mode = 'TRG' # 1) 'PSK'; 2) 'PCC'; 3) 'KNN'; 4) 'TRG'; 5) 'AWMF'.
edge_weight_mode = 'default' # 1) 'default'; 2) 'learnable'.
graph_fusion_mode = 'default' # 1) 'default'; 2) 'learnable'.
gnn_mode = 'GATv2Conv'
# 空间域： 1）GATConv (2017); 2) TransformerConv (2020); 3) GATv2Conv (2021); 4) SuperGATConv (2021)
# 谱域： 5）GCNConv (2016); 6) ChebConv (2016);  7) LEConv (2019); 8)SSGConv (2021).
## 3.3. 固定的参数 
readout_mode = 'mean' # 1) 'max'; 2) 'mean'; 3) 'fc‘.
## 3.4. 数据设置
#  psk邻接矩阵的路径: 获取当前文件所在目录os.getcwd()的上一级目录os.path.dirname(),即’models'，再进入'models\commonBlocks'
psk_path = os.path.join(os.path.dirname(os.getcwd()) , 'models', 'commonBlocks')
#  edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set': 计算pcc_adj和kgnn_adj
pcc_kgnn_adjs = None

### 4. 模型训练和测试的参数设置
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
## 4.1. 迭代次数、初始学习率、优化器、学习率衰减、早停和损失函数的设置
# 1)'epoch': int, 最大训练轮数
# 2)'initial_lr': 初始学习率，默认0.01
# 3)'optimizer': 优化器，默认'Adam'，one of ['Adam', 'RMSprop']
# 4)'lr_scheduler': 学习率衰减，scheduler_type: one of ['None', 'StepLR', 'MultiStepLR', 'ExponentialLR', AutoWarmupLR', 
#                                                   'GradualWarmupLR' ,'ReduceLROnPlateau']
# 5)'early_stopping': 早停
# 6)'criterion': 'loss_type', 默认'CE'('CrossEntropy')，one of ['CE', 'WeightedCE', 'FL'('FocalLoss'), 'WeightedFL', 'AttenuationWeightedCE', 'AttenuationWeightedFL']
#                'modify_type': 'exponent' / 'linear'
max_epoch = 100
callbacks = {'epoch': max_epoch,
             'initial_lr': 0.01,
             'optimizer': 'Adam',
             'lr_scheduler': {'scheduler_type': 'GradualWarmupLR',
                              'params':{
                                  'StepLR':{'step_size': int(0.2*max_epoch), 'gamma': np.sqrt(0.1)},
                                  'MultiStepLR':{'milestones': [int(0.2*max_epoch), int(0.4*max_epoch), int(0.6*max_epoch), int(0.8*max_epoch)], 
                                                 'gamma': np.sqrt(0.1)},
                                  'ExponentialLR':{'gamma': 0.9},
                                  'AutoWarmupLR':{'num_warm': 10},
                                  'GradualWarmupLR':{'multiplier': 1, 'total_epoch': 10},
                                  'ReduceLROnPlateau':{'mode': 'max', 'factor': np.sqrt(0.1),
                                                       'patience': 5, 'verbose': False,
                                                       'threshold': 0.0001, 'min_lr': 0.00001},
                              }
                              },
             'early_stopping': {'use_es': True, 'params':{'patience': 20, 'verbose': False, 'delta': 0.0001}},
             'criterion': {'loss_type': 'AttenuationWeightedFL', 'params':{'modify_type': 'exponent', 'exponent_factor': 5}}}

## 4.2. 训练和测试过程中的画图和保存设置
# 1) use_tqdm: 为True使用进度条在进度条内打印输出，为False使用hiddenlayer的History打印输出
# 2) train_plot: 是否使用hiddenlayer的Canvas为训练过程画图
# 3) print_interval: 多少个epoch打印一次输出或更新一次tqdm
# 4) model_eval: 使用验证集和测试集评估模型前是否打开model.eval()
# 5) test_metrics: list, 整个训练结束后在测试集上评估的指标 ['accuracy', 'precision', 'recall', 'f1', 'specificity', 'npv']
# 6) confusion_matrix: dist, 参考'metrics/get_test_metrics函数'. cmap: 'YlGnBu' / 'Blues'
#                     'show_type':  {'cm', 'normalized_cm', 'all'}
# 7) tsne_visualization: dist, 参考'visualization/tsne函数'
#    'show_type':  'train_set' / 'test_set' / 'all', 'feature_type': 'CNN' / 'Linear'
# 8）HMM模型for步态识别。步态分类任务的数据集划分方式 gait_dataset_divide_mode： ['random', 'group_fix', 'group_random']
train_test_utils = {'use_tqdm': False,
                    'train_plot': False,
                    'print_interval': 5,
                    'model_eval': {'valid': True, 'test': True},
                    'test_metrics': ['accuracy', 'precision', 'recall', 'f1', 'specificity', 'npv'],
                    'confusion_matrix': {'get_cm': False, 'params':{'show_type': 'all', 'plot': True, 'save_fig': True,
                                                                   'save_results': True, 'cmap': 'YlGnBu'}},
                    'tsne_visualization': {'get_tsne': False, 'params': {'show_type': 'all', 'feature_type': 'CNN', 
                                                                        'save_results': True, 'save_fig': True}},
                   'hmm_tools': {'gait': gait_or_motion, 'gait_dataset_divide_mode': 'group_random', 'use_hmm': True}
                    }            

## 1. 以下为测试TRG构图方式的代码

In [ ]:
## 1. 以下为测试TRGRatio的代码
"""
total_exp_time = 5
gait_or_motion = 'motion' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
conv_type = 'Conv'
edge_gen_mode = 'TRG'
edge_weight_mode = 'default'
gnn_mode = 'GATv2Conv'
callbacks['criterion']['loss_type'] = 'AttenuationWeightedCE'

settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}

trg_edge_connect_modes = ['graph_fix', 'node_fix', 'node_random']
max_esr_ratios = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# max_trg_edge_number = [1, 2, 3, 4, 6, 7, 9, 10, 12, 13]

for trg_edge_connect_mode in trg_edge_connect_modes:
    params['trg_edge_connect_mode'] = trg_edge_connect_mode
    
    for max_esr_ratio in max_esr_ratios:
        params['max_esr_ratio'] = max_esr_ratio
        model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                     conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
        model_name = model.get_model_name()
        printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
        basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '4-GNNs-TRGTest', trg_edge_connect_mode, str(max_esr_ratio))
        ## 开始训练和测试
        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
            save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
            print('保存结果的绝对路径： ', save_path['absolute_path'])
            print('保存结果的相对路径： ', save_path['relative_path'])
            
            callbacks['weights'] = class_weights
            train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
            
            for exp_tim in range(total_exp_time):
                clear()
                current_exp_time = exp_tim + 1
                settings_dict['current_exp_time'] = current_exp_time
                printlog(info='当前模型：%s, trg_edge_connect_mode: %s'% (model_name, params['trg_edge_connect_mode']), time=True, line_break=False)
                print('max_esr_ratio: %.2f, max_trg_edge_number: %d'% (params['max_esr_ratio'], math.floor(params['max_esr_ratio']*node_amount)))
                
                printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                if gait_or_motion == 'gait':
                    print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                     conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                model.double()
                model.to(device=device)
                train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                
        # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
        # 新增一个记录所有受试者的所有测试结果的df1
        df1_metrics = []
        df2_metrics_mean = []
        df2_metrics_std = []
        printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
        basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '4-GNNs-TRGTest', trg_edge_connect_mode, str(max_esr_ratio))
        
        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
            metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
            # 判断文件是否存在
            if not os.path.exists(metrics_file_name):
                print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
            else:
                # 读取每个受试者的test_metrics
                print("读取受试者：%s 的test_metrics: " %subject)
                df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                # ignore_index=True参数用于重置索引，以确保索引是连续的
                df1_metrics.extend(df.T.values[:-2, :])
                df2_metrics_mean.append(df.T.values[-2, :])
                df2_metrics_std.append(df.T.values[-1, :])
            
        printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
        df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
        mean_row.index = ['mean']  # 设置索引名称
        df1 = pd.concat([df1, mean_row])
        # 计算标准差并添加到DataFrame
        std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
        std_row.index = ['std']  # 设置索引名称
        df1 = pd.concat([df1, std_row]).round(3)
        # 保存df1  
        dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
        df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
        df1.to_csv(df1_save_name, index=True)
        
        printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
        # 保存df2
        df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
        df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
        df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
        df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
        std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
        df2.loc['mean'] = mean_row
        df2.loc['std'] = std_row
        df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
        df2.to_csv(df2_save_name, index=True)
        
        if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
            ## test_metrics——hmm.csv处理
            # 新增一个记录所有受试者的所有测试结果的df1
            df1_metrics = []
            df2_metrics_mean = []
            df2_metrics_std = []
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                # 判断文件是否存在
                if not os.path.exists(metrics_file_name):
                    print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                else:
                    # 读取每个受试者的test_metrics
                    print("读取受试者：%s 的test_metrics_hmm: " %subject)
                    df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                    # ignore_index=True参数用于重置索引，以确保索引是连续的
                    df1_metrics.extend(df.T.values[:-2, :])
                    df2_metrics_mean.append(df.T.values[-2, :])
                    df2_metrics_std.append(df.T.values[-1, :])
            
            printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
            df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
            mean_row.index = ['mean']  # 设置索引名称
            df1 = pd.concat([df1, mean_row])
            # 计算标准差并添加到DataFrame
            std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
            std_row.index = ['std']  # 设置索引名称
            df1 = pd.concat([df1, std_row]).round(3)
            # 保存df1  
            dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
            df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
            df1.to_csv(df1_save_name, index=True)
            
            printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
            # 保存df2
            df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
            df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
            df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
            df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
            std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
            df2.loc['mean'] = mean_row
            df2.loc['std'] = std_row
            df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
            df2.to_csv(df2_save_name, index=True)
"""

## 2. 以下为测试gnn_modes的代码

In [ ]:
## 2. 以下为测试gnn_modes的代码
# motion任务：conv_type = 'Conv'; params['max_esr_ratio'] = 0.1; edge_gen_mode = 'TRG'
# gait任务：conv_type = 'Conv'; params['max_esr_ratio'] = 0.2; edge_gen_mode = 'TRG'
# edge_weight_mode = 'default' (for GCNs和GATs) / 'learnable' (for GCNs)
"""
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}
conv_type = 'Conv'
edge_gen_mode = 'TRG'
gnn_modes = ['GATConv', 'TransformerConv', 'GATv2Conv', 'SuperGATConv', 'GCNConv', 'ChebConv', 'LEConv', 'SSGConv']
edge_weight_mode = 'default' # 'default' (for GCNs和GATs) / 'learnable' (for GCNs)
    
for gnn_mode in gnn_modes:
    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
    model_name = model.get_model_name()
    printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
    basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '5-GNNs-GNNModesTest')
    ## 开始训练和测试
    for subject_order in range(len(subjects_list_global)):
        subject = subjects_list_global[subject_order]
        file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
        save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
        print('保存结果的绝对路径： ', save_path['absolute_path'])
        print('保存结果的相对路径： ', save_path['relative_path'])
        
        callbacks['weights'] = class_weights
        train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
        
        for exp_tim in range(total_exp_time):
            clear()
            current_exp_time = exp_tim + 1
            settings_dict['current_exp_time'] = current_exp_time
            printlog(info='当前模型：%s, trg_edge_mode构建方式: %s'% (model_name, params['trg_edge_connect_mode']), time=True, line_break=False)
            print('max_esr_ratio: %.2f, max_trg_edge_number: %d'% (params['max_esr_ratio'], math.floor(params['max_esr_ratio']*node_amount)))
            
            printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
            printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
            train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
            if gait_or_motion == 'gait':
                print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
            print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
            model.double()
            model.to(device=device)
            train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
            
    # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
    # 新增一个记录所有受试者的所有测试结果的df1
    df1_metrics = []
    df2_metrics_mean = []
    df2_metrics_std = []
    printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
    basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '5-GNNs-GNNModesTest')
    
    for subject_order in range(len(subjects_list_global)):
        subject = subjects_list_global[subject_order]
        metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
        metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
        # 判断文件是否存在
        if not os.path.exists(metrics_file_name):
            print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
        else:
            # 读取每个受试者的test_metrics
            print("读取受试者：%s 的test_metrics: " %subject)
            df = pd.read_csv(metrics_file_name, header=0, index_col=0)
            # ignore_index=True参数用于重置索引，以确保索引是连续的
            df1_metrics.extend(df.T.values[:-2, :])
            df2_metrics_mean.append(df.T.values[-2, :])
            df2_metrics_std.append(df.T.values[-1, :])
        
    printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
    df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
    # 计算平均值并添加到DataFrame
    mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
    mean_row.index = ['mean']  # 设置索引名称
    df1 = pd.concat([df1, mean_row])
    # 计算标准差并添加到DataFrame
    std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
    std_row.index = ['std']  # 设置索引名称
    df1 = pd.concat([df1, std_row]).round(3)
    # 保存df1  
    dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
    df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
    df1.to_csv(df1_save_name, index=True)
    
    printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
    # 保存df2
    df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
    df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
    df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
    df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
    # 计算平均值并添加到DataFrame
    mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
    std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
    df2.loc['mean'] = mean_row
    df2.loc['std'] = std_row
    df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
    df2.to_csv(df2_save_name, index=True)
    
    if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
        ## test_metrics——hmm.csv处理
        # 新增一个记录所有受试者的所有测试结果的df1
        df1_metrics = []
        df2_metrics_mean = []
        df2_metrics_std = []
        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
            metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
            # 判断文件是否存在
            if not os.path.exists(metrics_file_name):
                print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
            else:
                # 读取每个受试者的test_metrics
                print("读取受试者：%s 的test_metrics_hmm: " %subject)
                df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                # ignore_index=True参数用于重置索引，以确保索引是连续的
                df1_metrics.extend(df.T.values[:-2, :])
                df2_metrics_mean.append(df.T.values[-2, :])
                df2_metrics_std.append(df.T.values[-1, :])
        
        printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
        df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
        mean_row.index = ['mean']  # 设置索引名称
        df1 = pd.concat([df1, mean_row])
        # 计算标准差并添加到DataFrame
        std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
        std_row.index = ['std']  # 设置索引名称
        df1 = pd.concat([df1, std_row]).round(3)
        # 保存df1  
        dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
        df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
        df1.to_csv(df1_save_name, index=True)
        
        printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
        # 保存df2
        df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
        df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
        df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
        df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
        std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
        df2.loc['mean'] = mean_row
        df2.loc['std'] = std_row
        df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
        df2.to_csv(df2_save_name, index=True)
"""

## 3. 以下为测试PSK图拓扑生成的代码

In [ ]:
## 3. 以下为测试PSK图拓扑生成的代码
"""
total_exp_time = 5
gait_or_motion = 'motion' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
conv_type = 'Conv'
edge_gen_mode = 'PSK'
edge_weight_mode = 'default'
gnn_modes = ['TransformerConv','LEConv']
# gnn_modes = ['LEConv']
callbacks['criterion']['loss_type'] = 'AttenuationWeightedCE'

settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}
  
for gnn_mode in gnn_modes:
    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
    model_name = model.get_model_name()
    printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
    basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '6-GNNs-PSKTest')
    ## 开始训练和测试
    for subject_order in range(len(subjects_list_global)):
        subject = subjects_list_global[subject_order]
        file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
        save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
        print('保存结果的绝对路径： ', save_path['absolute_path'])
        print('保存结果的相对路径： ', save_path['relative_path'])
        
        callbacks['weights'] = class_weights
        train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
        
        for exp_tim in range(total_exp_time):
            clear()
            current_exp_time = exp_tim + 1
            settings_dict['current_exp_time'] = current_exp_time
            printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
            printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
            
            printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
            train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
            if gait_or_motion == 'gait':
                print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
            print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
            model.double()
            model.to(device=device)
            train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
            
    # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
    # 新增一个记录所有受试者的所有测试结果的df1
    df1_metrics = []
    df2_metrics_mean = []
    df2_metrics_std = []
    printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
    basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '6-GNNs-PSKTest')
    
    for subject_order in range(len(subjects_list_global)):
        subject = subjects_list_global[subject_order]
        metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
        metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
        # 判断文件是否存在
        if not os.path.exists(metrics_file_name):
            print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
        else:
            # 读取每个受试者的test_metrics
            print("读取受试者：%s 的test_metrics: " %subject)
            df = pd.read_csv(metrics_file_name, header=0, index_col=0)
            # ignore_index=True参数用于重置索引，以确保索引是连续的
            df1_metrics.extend(df.T.values[:-2, :])
            df2_metrics_mean.append(df.T.values[-2, :])
            df2_metrics_std.append(df.T.values[-1, :])
        
    printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
    df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
    # 计算平均值并添加到DataFrame
    mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
    mean_row.index = ['mean']  # 设置索引名称
    df1 = pd.concat([df1, mean_row])
    # 计算标准差并添加到DataFrame
    std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
    std_row.index = ['std']  # 设置索引名称
    df1 = pd.concat([df1, std_row]).round(3)
    # 保存df1  
    dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
    df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
    df1.to_csv(df1_save_name, index=True)
    
    printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
    # 保存df2
    df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
    df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
    df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
    df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
    # 计算平均值并添加到DataFrame
    mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
    std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
    df2.loc['mean'] = mean_row
    df2.loc['std'] = std_row
    df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
    df2.to_csv(df2_save_name, index=True)
    
    if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
        ## test_metrics——hmm.csv处理
        # 新增一个记录所有受试者的所有测试结果的df1
        df1_metrics = []
        df2_metrics_mean = []
        df2_metrics_std = []
        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
            metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
            # 判断文件是否存在
            if not os.path.exists(metrics_file_name):
                print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
            else:
                # 读取每个受试者的test_metrics
                print("读取受试者：%s 的test_metrics_hmm: " %subject)
                df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                # ignore_index=True参数用于重置索引，以确保索引是连续的
                df1_metrics.extend(df.T.values[:-2, :])
                df2_metrics_mean.append(df.T.values[-2, :])
                df2_metrics_std.append(df.T.values[-1, :])
        
        printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
        df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
        mean_row.index = ['mean']  # 设置索引名称
        df1 = pd.concat([df1, mean_row])
        # 计算标准差并添加到DataFrame
        std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
        std_row.index = ['std']  # 设置索引名称
        df1 = pd.concat([df1, std_row]).round(3)
        # 保存df1  
        dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
        df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
        df1.to_csv(df1_save_name, index=True)
        
        printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
        # 保存df2
        df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
        df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
        df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
        df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
        std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
        df2.loc['mean'] = mean_row
        df2.loc['std'] = std_row
        df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
        df2.to_csv(df2_save_name, index=True)
"""

## 4. 以下为测试PCC构图方式的代码

In [ ]:
## 4. 以下为测试PCC构图的代码
"""
total_exp_time = 5
network_branch = 1
gait_or_motion = 'gait' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
conv_type = 'Conv'
edge_gen_mode = 'PCC'
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
gnn_modes = ['TransformerConv','LEConv']

# pcc_kgnn_gen_modes = ['batch_cnn_feature', 'batch_train_data', 'train_set']
pcc_kgnn_gen_modes = ['train_set']
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}
callbacks['criterion']['loss_type'] = 'AttenuationWeightedCE'      

for pcc_kgnn_gen_mode in pcc_kgnn_gen_modes:
    params['pcc_kgnn_gen_mode'] = pcc_kgnn_gen_mode
    for gnn_mode in gnn_modes:
        if pcc_kgnn_gen_mode == 'train_set':
            pcc_act_thrs = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
            if gnn_mode == 'LEConv':
                edge_weight_mode = 'learnable' 
        else:
            edge_weight_mode = 'default'
            pcc_act_thrs = [0.9, 0.8, 0.7, 0.6, 0.5]
            
        for pcc_act_thr in pcc_act_thrs:
            params['pcc_act_thr'] = pcc_act_thr
            params['pcc_kgnn_adjs'] = None
            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
            model_name = model.get_model_name()
            printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
            basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '7-GNNs-PCCTest', pcc_kgnn_gen_mode, str(pcc_act_thr))
            ## 开始训练和测试
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
                save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
                print('保存结果的绝对路径： ', save_path['absolute_path'])
                print('保存结果的相对路径： ', save_path['relative_path'])
                
                callbacks['weights'] = class_weights
                train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
                
                for exp_tim in range(total_exp_time):
                    clear()
                    current_exp_time = exp_tim + 1
                    settings_dict['current_exp_time'] = current_exp_time
                    printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
                    print('edge生成方式: %s, pcc_kgnn_gen_mode: %s '% (pcc_kgnn_gen_mode, edge_gen_mode))
                    print('pcc_act_thr: %.2f'% params['pcc_act_thr'])
                    
                    printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                    printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                    train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    if gait_or_motion == 'gait':
                        print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                    if edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set':
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    elif edge_gen_mode == 'AWMF' and ('PCC' in params['AWMF_type'] or 'KNN' in params['AWMF_type']):
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    else:
                        params['pcc_kgnn_adjs'] = None
                    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                    model.double()
                    model.to(device=device)
                    train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                    
            # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
            # 新增一个记录所有受试者的所有测试结果的df1
            df1_metrics = []
            df2_metrics_mean = []
            df2_metrics_std = []
            printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
            basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '7-GNNs-PCCTest', pcc_kgnn_gen_mode, str(pcc_act_thr))
            
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
                # 判断文件是否存在
                if not os.path.exists(metrics_file_name):
                    print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                else:
                    # 读取每个受试者的test_metrics
                    print("读取受试者：%s 的test_metrics: " %subject)
                    df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                    # ignore_index=True参数用于重置索引，以确保索引是连续的
                    df1_metrics.extend(df.T.values[:-2, :])
                    df2_metrics_mean.append(df.T.values[-2, :])
                    df2_metrics_std.append(df.T.values[-1, :])
                
            printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
            df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
            mean_row.index = ['mean']  # 设置索引名称
            df1 = pd.concat([df1, mean_row])
            # 计算标准差并添加到DataFrame
            std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
            std_row.index = ['std']  # 设置索引名称
            df1 = pd.concat([df1, std_row]).round(3)
            # 保存df1  
            dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
            df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
            df1.to_csv(df1_save_name, index=True)
            
            printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
            # 保存df2
            df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
            df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
            df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
            df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
            std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
            df2.loc['mean'] = mean_row
            df2.loc['std'] = std_row
            df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
            df2.to_csv(df2_save_name, index=True)
            
            if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
                ## test_metrics——hmm.csv处理
                # 新增一个记录所有受试者的所有测试结果的df1
                df1_metrics = []
                df2_metrics_mean = []
                df2_metrics_std = []
                for subject_order in range(len(subjects_list_global)):
                    subject = subjects_list_global[subject_order]
                    metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                    metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                    # 判断文件是否存在
                    if not os.path.exists(metrics_file_name):
                        print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                    else:
                        # 读取每个受试者的test_metrics
                        print("读取受试者：%s 的test_metrics_hmm: " %subject)
                        df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                        # ignore_index=True参数用于重置索引，以确保索引是连续的
                        df1_metrics.extend(df.T.values[:-2, :])
                        df2_metrics_mean.append(df.T.values[-2, :])
                        df2_metrics_std.append(df.T.values[-1, :])
                
                printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                mean_row.index = ['mean']  # 设置索引名称
                df1 = pd.concat([df1, mean_row])
                # 计算标准差并添加到DataFrame
                std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                std_row.index = ['std']  # 设置索引名称
                df1 = pd.concat([df1, std_row]).round(3)
                # 保存df1  
                dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
                df1.to_csv(df1_save_name, index=True)
                
                printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                # 保存df2
                df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                df2.loc['mean'] = mean_row
                df2.loc['std'] = std_row
                df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
                df2.to_csv(df2_save_name, index=True)
"""       

## 5. 以下为测试KNN构图方式的代码

In [ ]:
## 5. 以下为测试KNNRatio的代码
# pip install torch-cluster
"""
total_exp_time = 5
network_branch = 1
gait_or_motion = 'gait' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
conv_type = 'Conv'
edge_gen_mode = 'KNN'
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.3, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.5, 'kgnn_act_thr': 0.5,
          'awmf_act_thr': 0.5, 'AWMF_type': ['PSK', 'TRG', 'PCC', 'KNN']}

# pcc_kgnn_gen_modes = ['batch_cnn_feature', 'batch_train_data', 'train_set']
pcc_kgnn_gen_modes = ['train_set']
kgnn_ratios = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
gnn_modes = ['TransformerConv','LEConv']
callbacks['criterion']['loss_type'] = 'AttenuationWeightedCE'

for pcc_kgnn_gen_mode in pcc_kgnn_gen_modes:
    params['pcc_kgnn_gen_mode'] = pcc_kgnn_gen_mode
    
    for gnn_mode in gnn_modes:
        if pcc_kgnn_gen_mode == 'train_set':
            kgnn_act_thrs = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
            if gnn_mode == 'LEConv':
                edge_weight_mode = 'learnable' 
        else:
            edge_weight_mode = 'default'
            kgnn_act_thrs = ['default_weight']
            
        for kgnn_act_thr in kgnn_act_thrs:
            params['kgnn_act_thr'] = kgnn_act_thr
            
            for kgnn_ratio in kgnn_ratios:
                params['kgnn_ratio'] = kgnn_ratio
                params['pcc_kgnn_adjs'] = None
                
                model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                             conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                model_name = model.get_model_name()
                printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
                basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '8-GNNs-KNNTest', pcc_kgnn_gen_mode,
                                               str(kgnn_act_thr), str(kgnn_ratio))
                ## 开始训练和测试
                for subject_order in range(len(subjects_list_global)):
                    subject = subjects_list_global[subject_order]
                    file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
                    save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
                    print('保存结果的绝对路径： ', save_path['absolute_path'])
                    print('保存结果的相对路径： ', save_path['relative_path'])
                    
                    callbacks['weights'] = class_weights
                    train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
                    
                    for exp_tim in range(total_exp_time):
                        clear()
                        current_exp_time = exp_tim + 1
                        settings_dict['current_exp_time'] = current_exp_time
                        printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
                        print('edge生成方式: %s, pcc_kgnn_gen_mode: %s'% (edge_gen_mode, params['pcc_kgnn_gen_mode']))
                        print('kgnn_act_thr: %s, kgnn_ratio: %.2f'% (str(params['kgnn_act_thr']), params['kgnn_ratio']))
                        
                        printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                        printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                        train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                        if gait_or_motion == 'gait':
                            print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                        print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                        if edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set':
                            params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                        elif edge_gen_mode == 'AWMF' and ('PCC' in params['AWMF_type'] or 'KNN' in params['AWMF_type']):
                            params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                        else:
                            params['pcc_kgnn_adjs'] = None
                        model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                             conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                        model.double()
                        model.to(device=device)
                        train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                        
                # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
                # 新增一个记录所有受试者的所有测试结果的df1
                df1_metrics = []
                df2_metrics_mean = []
                df2_metrics_std = []
                printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
                basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '8-GNNs-KNNTest', pcc_kgnn_gen_mode,
                                               str(kgnn_act_thr), str(kgnn_ratio))
                
                for subject_order in range(len(subjects_list_global)):
                    subject = subjects_list_global[subject_order]
                    metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                    metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
                    # 判断文件是否存在
                    if not os.path.exists(metrics_file_name):
                        print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                    else:
                        # 读取每个受试者的test_metrics
                        print("读取受试者：%s 的test_metrics: " %subject)
                        df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                        # ignore_index=True参数用于重置索引，以确保索引是连续的
                        df1_metrics.extend(df.T.values[:-2, :])
                        df2_metrics_mean.append(df.T.values[-2, :])
                        df2_metrics_std.append(df.T.values[-1, :])
                    
                printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                mean_row.index = ['mean']  # 设置索引名称
                df1 = pd.concat([df1, mean_row])
                # 计算标准差并添加到DataFrame
                std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                std_row.index = ['std']  # 设置索引名称
                df1 = pd.concat([df1, std_row]).round(3)
                # 保存df1  
                dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
                df1.to_csv(df1_save_name, index=True)
                
                printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                # 保存df2
                df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                df2.loc['mean'] = mean_row
                df2.loc['std'] = std_row
                df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
                df2.to_csv(df2_save_name, index=True)
                
                if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
                    ## test_metrics——hmm.csv处理
                    # 新增一个记录所有受试者的所有测试结果的df1
                    df1_metrics = []
                    df2_metrics_mean = []
                    df2_metrics_std = []
                    for subject_order in range(len(subjects_list_global)):
                        subject = subjects_list_global[subject_order]
                        metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                        metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                        # 判断文件是否存在
                        if not os.path.exists(metrics_file_name):
                            print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                        else:
                            # 读取每个受试者的test_metrics
                            print("读取受试者：%s 的test_metrics_hmm: " %subject)
                            df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                            # ignore_index=True参数用于重置索引，以确保索引是连续的
                            df1_metrics.extend(df.T.values[:-2, :])
                            df2_metrics_mean.append(df.T.values[-2, :])
                            df2_metrics_std.append(df.T.values[-1, :])
                    
                    printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                    df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                    # 计算平均值并添加到DataFrame
                    mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                    mean_row.index = ['mean']  # 设置索引名称
                    df1 = pd.concat([df1, mean_row])
                    # 计算标准差并添加到DataFrame
                    std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                    std_row.index = ['std']  # 设置索引名称
                    df1 = pd.concat([df1, std_row]).round(3)
                    # 保存df1  
                    dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                    df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
                    df1.to_csv(df1_save_name, index=True)
                    
                    printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                    # 保存df2
                    df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                    df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                    df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                    df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                    # 计算平均值并添加到DataFrame
                    mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                    std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                    df2.loc['mean'] = mean_row
                    df2.loc['std'] = std_row
                    df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
                    df2.to_csv(df2_save_name, index=True)
"""

## 6. 以下为测试AWMF构图方式的代码

In [ ]:
## 6. 以下为测试AWMF构图的代码
"""
# 1) 固定参数
total_exp_time = 5
network_branch = 1
gait_or_motion = 'gait' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
conv_type = 'Conv'
edge_gen_mode = 'AWMF'
gnn_modes = ['LEConv', 'GATv2Conv', 'TransformerConv']
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.2, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.3, 'kgnn_act_thr': 0.3,
          'awmf_act_thr': 0.5, 'AWMF_type': ['TRG', 'PSK', 'PCC', 'KNN']}
callbacks['criterion']['loss_type'] = 'AttenuationWeightedCE'
# 2) 调节参数
graph_fusion_modes = ['default', 'learnable']
# AWMF_types = [['TRG', 'PSK'], ['TRG', 'PCC'], ['TRG','KNN'], ['PCC', 'PSK'], ['PCC', 'KNN'], ['PSK','KNN'], 
#               ['TRG', 'PSK', 'PCC'], ['TRG', 'PSK', 'KNN'], ['TRG', 'PCC', 'KNN'], ['PSK', 'PCC', 'KNN'], 
#               ['TRG', 'PSK', 'PCC', 'KNN']]

AWMF_types = [['TRG', 'PSK'], ['TRG','KNN'], ['PSK','KNN'], 
              ['TRG', 'PSK', 'PCC'], ['TRG', 'PSK', 'KNN']]

for gnn_mode in gnn_modes:
    if gnn_mode == 'LEConv':
        edge_weight_modes = ['default', 'learnable'] # 这里AWMF的edge_weight和其他图拓扑的可训练的边权不同，只是一个融合后计算出的系数
    elif gnn_mode in ['GATv2Conv', 'TransformerConv']:
        edge_weight_modes = ['default']
    else: 
        raise ValueError('gnn_mode only support "LEConv", "GATv2Conv", "TransformerConv" ')
    
    for graph_fusion_mode in graph_fusion_modes: 
        for edge_weight_mode in edge_weight_modes:
            for index in range(len(AWMF_types)):
                AWMF_type = AWMF_types[index]
                AWMF_type_str = AWMF_type[0]
                for k in range(1, len(AWMF_type)):
                    AWMF_type_str = AWMF_type_str + '-' + AWMF_type[k]
                params['AWMF_type'] = AWMF_type
                awmf_act_thrs = [i / len(AWMF_type) for i in range(1, len(AWMF_type))]
                
                for awmf_act_thr in awmf_act_thrs:
                    params['pcc_kgnn_adjs'] = None
                    params['awmf_act_thr'] = float(awmf_act_thr - 0.01)
                    
                    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                    model_name = model.get_model_name()
                    printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
                    basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '9-GNNs-AWMFTest', graph_fusion_mode, edge_weight_mode, AWMF_type_str, str(awmf_act_thr))
                    ## 开始训练和测试
                    for subject_order in range(len(subjects_list_global)):
                        subject = subjects_list_global[subject_order]
                        file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
                        save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
                        print('保存结果的绝对路径： ', save_path['absolute_path'])
                        print('保存结果的相对路径： ', save_path['relative_path'])
                        
                        callbacks['weights'] = class_weights
                        train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
                        
                        for exp_tim in range(total_exp_time):
                            clear()
                            current_exp_time = exp_tim + 1
                            settings_dict['current_exp_time'] = current_exp_time
                            printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
                            print('edg_gen_mode: %s, graph_fusion_mode: %s, edge_weight_mode: %s '% (edge_gen_mode, graph_fusion_mode, edge_weight_mode))
                            print('AWMF_type: %s, awmf_act_thr: %.4f'% (AWMF_type_str, awmf_act_thr))
                            
                            printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                            printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                            train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                            if gait_or_motion == 'gait':
                                print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                            print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                            if edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set':
                                params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                            elif edge_gen_mode == 'AWMF' and ('PCC' in params['AWMF_type'] or 'KNN' in params['AWMF_type']):
                                params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                            else:
                                params['pcc_kgnn_adjs'] = None
                            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                                 conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                            model.double()
                            model.to(device=device)
                            train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                            
                    # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
                    # 新增一个记录所有受试者的所有测试结果的df1
                    df1_metrics = []
                    df2_metrics_mean = []
                    df2_metrics_std = []
                    printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
                    basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '9-GNNs-AWMFTest', graph_fusion_mode, edge_weight_mode, AWMF_type_str, str(awmf_act_thr))
                    
                    for subject_order in range(len(subjects_list_global)):
                        subject = subjects_list_global[subject_order]
                        metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                        metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
                        # 判断文件是否存在
                        if not os.path.exists(metrics_file_name):
                            print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                        else:
                            # 读取每个受试者的test_metrics
                            print("读取受试者：%s 的test_metrics: " %subject)
                            df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                            # ignore_index=True参数用于重置索引，以确保索引是连续的
                            df1_metrics.extend(df.T.values[:-2, :])
                            df2_metrics_mean.append(df.T.values[-2, :])
                            df2_metrics_std.append(df.T.values[-1, :])
                        
                    printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                    df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                    # 计算平均值并添加到DataFrame
                    mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                    mean_row.index = ['mean']  # 设置索引名称
                    df1 = pd.concat([df1, mean_row])
                    # 计算标准差并添加到DataFrame
                    std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                    std_row.index = ['std']  # 设置索引名称
                    df1 = pd.concat([df1, std_row]).round(3)
                    # 保存df1  
                    dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                    df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
                    df1.to_csv(df1_save_name, index=True)
                    
                    printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                    # 保存df2
                    df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                    df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                    df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                    df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                    # 计算平均值并添加到DataFrame
                    mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                    std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                    df2.loc['mean'] = mean_row
                    df2.loc['std'] = std_row
                    df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
                    df2.to_csv(df2_save_name, index=True)
                    
                    if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
                        ## test_metrics——hmm.csv处理
                        # 新增一个记录所有受试者的所有测试结果的df1
                        df1_metrics = []
                        df2_metrics_mean = []
                        df2_metrics_std = []
                        for subject_order in range(len(subjects_list_global)):
                            subject = subjects_list_global[subject_order]
                            metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                            metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                            # 判断文件是否存在
                            if not os.path.exists(metrics_file_name):
                                print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                            else:
                                # 读取每个受试者的test_metrics
                                print("读取受试者：%s 的test_metrics_hmm: " %subject)
                                df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                                # ignore_index=True参数用于重置索引，以确保索引是连续的
                                df1_metrics.extend(df.T.values[:-2, :])
                                df2_metrics_mean.append(df.T.values[-2, :])
                                df2_metrics_std.append(df.T.values[-1, :])
                        
                        printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                        df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                        # 计算平均值并添加到DataFrame
                        mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                        mean_row.index = ['mean']  # 设置索引名称
                        df1 = pd.concat([df1, mean_row])
                        # 计算标准差并添加到DataFrame
                        std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                        std_row.index = ['std']  # 设置索引名称
                        df1 = pd.concat([df1, std_row]).round(3)
                        # 保存df1  
                        dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                        df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
                        df1.to_csv(df1_save_name, index=True)
                        
                        printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                        # 保存df2
                        df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                        df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                        df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                        df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                        # 计算平均值并添加到DataFrame
                        mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                        std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                        df2.loc['mean'] = mean_row
                        df2.loc['std'] = std_row
                        df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
                        df2.to_csv(df2_save_name, index=True)
"""     

## 7. 以下为测试KNN和AWMF构图，单双分支的LEConv在不同损失函数中的差异代码

In [ ]:
## 7. 以下为测试KNN和AWMF构图，单双分支的LEConv在不同损失函数中的差异代码
"""
gait_or_motion = 'gait' # 'gait', 'motion'
motion_type = 'WAK' #'WAK', 'UPS', 'DNS'
total_exp_time = 5
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.2, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.3, 'kgnn_act_thr': 0.3,
          'awmf_act_thr': 1/3, 'AWMF_type': ['TRG', 'PSK', 'KNN']}
conv_type = 'Conv' 
graph_fusion_mode = 'default'
gnn_mode = 'LEConv'

# 要更改的参数
edge_gen_modes = ['KNN', 'AWMF']
network_branchs = [1, 2]
if gait_or_motion == 'motion':
    criterions = ['FL']
    params['max_esr_ratio'] = 0.1
elif gait_or_motion == 'gait':
    criterions = ['CE', 'FL', 'WeightedCE', 'AttenuationWeightedCE']
    params['max_esr_ratio'] = 0.2
else:
    raise ValueError('gait_or_motion must be "gait" or "motion"')

for network_branch in network_branchs:
    for edge_gen_mode in edge_gen_modes:
        if edge_gen_mode == 'KNN':
            edge_weight_mode = 'learnable'
        else:
            edge_weight_mode = 'default'
            
        for criterion in criterions:
            callbacks['criterion']['loss_type'] = criterion
            params['pcc_kgnn_adjs'] = None
            params['awmf_act_thr'] = float(params['awmf_act_thr'] - 0.01)
            
            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
            model_name = model.get_model_name()
            printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
            basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '10-GNNs-LossFunctionTest', criterion)
            ## 开始训练和测试
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
                save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
                print('保存结果的绝对路径： ', save_path['absolute_path'])
                print('保存结果的相对路径： ', save_path['relative_path'])
                
                callbacks['weights'] = class_weights
                train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
                
                for exp_tim in range(total_exp_time):
                    clear()
                    current_exp_time = exp_tim + 1
                    settings_dict['current_exp_time'] = current_exp_time
                    printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
                    print('损失函数: %s'% criterion)
                    print('edge_gen_mode: %s, edge_weight_mode: %s'% (edge_gen_mode, edge_weight_mode))
                    if edge_gen_mode == 'AWMF':
                        print('AWMF-graph_fusion_mode: %s'% graph_fusion_mode)
                        
                    printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                    printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                    train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    if gait_or_motion == 'gait':
                        print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                    if edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set':
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    elif edge_gen_mode == 'AWMF' and ('PCC' in params['AWMF_type'] or 'KNN' in params['AWMF_type']):
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    else:
                        params['pcc_kgnn_adjs'] = None
                        
                    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                    model.double()
                    model.to(device=device)
                    train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                    
            # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
            # 新增一个记录所有受试者的所有测试结果的df1
            df1_metrics = []
            df2_metrics_mean = []
            df2_metrics_std = []
            printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
            basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '10-GNNs-LossFunctionTest',criterion)
            
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
                # 判断文件是否存在
                if not os.path.exists(metrics_file_name):
                    print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                else:
                    # 读取每个受试者的test_metrics
                    print("读取受试者：%s 的test_metrics: " %subject)
                    df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                    # ignore_index=True参数用于重置索引，以确保索引是连续的
                    df1_metrics.extend(df.T.values[:-2, :])
                    df2_metrics_mean.append(df.T.values[-2, :])
                    df2_metrics_std.append(df.T.values[-1, :])
                
            printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
            df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
            mean_row.index = ['mean']  # 设置索引名称
            df1 = pd.concat([df1, mean_row])
            # 计算标准差并添加到DataFrame
            std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
            std_row.index = ['std']  # 设置索引名称
            df1 = pd.concat([df1, std_row]).round(3)
            # 保存df1  
            dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
            df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
            df1.to_csv(df1_save_name, index=True)
            
            printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
            # 保存df2
            df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
            df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
            df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
            df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
            std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
            df2.loc['mean'] = mean_row
            df2.loc['std'] = std_row
            df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
            df2.to_csv(df2_save_name, index=True)
            
            if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
                ## test_metrics——hmm.csv处理
                # 新增一个记录所有受试者的所有测试结果的df1
                df1_metrics = []
                df2_metrics_mean = []
                df2_metrics_std = []
                for subject_order in range(len(subjects_list_global)):
                    subject = subjects_list_global[subject_order]
                    metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                    metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                    # 判断文件是否存在
                    if not os.path.exists(metrics_file_name):
                        print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                    else:
                        # 读取每个受试者的test_metrics
                        print("读取受试者：%s 的test_metrics_hmm: " %subject)
                        df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                        # ignore_index=True参数用于重置索引，以确保索引是连续的
                        df1_metrics.extend(df.T.values[:-2, :])
                        df2_metrics_mean.append(df.T.values[-2, :])
                        df2_metrics_std.append(df.T.values[-1, :])
                
                printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                mean_row.index = ['mean']  # 设置索引名称
                df1 = pd.concat([df1, mean_row])
                # 计算标准差并添加到DataFrame
                std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                std_row.index = ['std']  # 设置索引名称
                df1 = pd.concat([df1, std_row]).round(3)
                # 保存df1  
                dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
                df1.to_csv(df1_save_name, index=True)
                
                printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                # 保存df2
                df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                df2.loc['mean'] = mean_row
                df2.loc['std'] = std_row
                df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
                df2.to_csv(df2_save_name, index=True)
"""

## 8. 以下为先进方法对比中，测试KNN和AWMF构图，双分支的LEConv在不同损失函数中的差异代码

In [ ]:
## 8. 以下为先进方法对比中，测试KNN和AWMF构图，双分支的LEConv在不同损失函数中的差异代码
total_exp_time = 5
network_branch = 2
settings_dict = {'total_exp_time': total_exp_time, 'network_branch': network_branch}
params = {'max_esr_ratio': 0.2, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 
          'pcc_kgnn_gen_mode': 'train_set', 'pcc_act_thr': 0.5, 'kgnn_ratio': 0.3, 'kgnn_act_thr': 0.3,
          'awmf_act_thr': 1/3, 'AWMF_type': ['TRG', 'PSK', 'KNN']}
conv_type = 'Conv' 
graph_fusion_mode = 'default'
gnn_mode = 'LEConv'

# 要更改的参数
data_params = {'gait_or_motion': ['motion', 'gait', 'gait', 'gait'],
               'motion_type': ['WAK', 'WAK', 'UPS', 'DNS']}
edge_gen_modes = ['KNN', 'AWMF']
criterions = ['CE', 'FL', 'WeightedCE', 'AttenuationWeightedCE']
train_test_utils['confusion_matrix']['get_cm'] = True
train_test_utils['tsne_visualization']['get_tsne'] = True

for index in range(len(data_params['gait_or_motion'])):
    gait_or_motion = data_params['gait_or_motion'][index]
    motion_type = data_params['motion_type'][index]
    if gait_or_motion == 'motion':
        # criterions = ['FL']
        params['max_esr_ratio'] = 0.1
    elif gait_or_motion == 'gait':
        # criterions = ['CE', 'FL', 'WeightedCE', 'AttenuationWeightedCE']
        params['max_esr_ratio'] = 0.2
    else:
        raise ValueError('gait_or_motion must be "gait" or "motion"')
    
    for edge_gen_mode in edge_gen_modes:
        if edge_gen_mode == 'KNN':
            edge_weight_mode = 'learnable'
        else:
            edge_weight_mode = 'default'
            
        for criterion in criterions:
            callbacks['criterion']['loss_type'] = criterion
            params['pcc_kgnn_adjs'] = None
            params['awmf_act_thr'] = float(params['awmf_act_thr'] - 0.01)
            
            model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
            model_name = model.get_model_name()
            printlog(info=get_print_info(gait_or_motion, motion_type, subjects_list_global), time=False, line_break=False)
            basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '11-Start-of-the-artMethodsComparison', criterion)
            # basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '11-Start-of-the-artMethodsComparison')
            # 开始训练和测试
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                file_name, class_weights, encoded_label_name, raw_label_type, true_labels  = get_fileName_weights(file_path, gait_or_motion, motion_type, subject, subjects_list_global)
                save_path = get_save_path(basic_save_path, gait_or_motion, motion_type, model_name, subject)
                print('保存结果的绝对路径： ', save_path['absolute_path'])
                print('保存结果的相对路径： ', save_path['relative_path'])
                
                callbacks['weights'] = class_weights
                train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
                
                for exp_tim in range(total_exp_time):
                    clear()
                    current_exp_time = exp_tim + 1
                    settings_dict['current_exp_time'] = current_exp_time
                    printlog(info='当前模型：%s'% model_name, time=True, line_break=False)
                    if gait_or_motion == 'motion':
                        print('gait_or_motion: %s'% gait_or_motion)
                    else:
                        print('gait_or_motion: %s, motion_type: %s'% (gait_or_motion, motion_type))
                    print('损失函数: %s'% criterion)
                    print('edge_gen_mode: %s, edge_weight_mode: %s'% (edge_gen_mode, edge_weight_mode))
                    if edge_gen_mode == 'AWMF':
                        print('AWMF-graph_fusion_mode: %s'% graph_fusion_mode)
                        
                    printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                    printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                    train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, data_list, feature_list, encoded_label_name, total_exp_time,gait_or_motion, current_exp_time, train_batch, test_batch, valid_batch, train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    if gait_or_motion == 'gait':
                        print('步态数据集划分方式： ', train_test_utils['hmm_tools']['gait_dataset_divide_mode'])
                    print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                    if edge_gen_mode in ['PCC', 'KNN'] and params['pcc_kgnn_gen_mode'] == 'train_set':
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    elif edge_gen_mode == 'AWMF' and ('PCC' in params['AWMF_type'] or 'KNN' in params['AWMF_type']):
                        params['pcc_kgnn_adjs'] = get_pcc_knn_adj_from_dataloader(train_loader, params, edge_gen_mode)
                    else:
                        params['pcc_kgnn_adjs'] = None
                        
                    model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                         conv_type, node_amount, gait_or_motion, motion_type, psk_path, params)
                    model.double()
                    model.to(device=device)
                    train_test_intra_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils, true_labels)
                    
            # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
            # 新增一个记录所有受试者的所有测试结果的df1
            df1_metrics = []
            df2_metrics_mean = []
            df2_metrics_std = []
            printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
            basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '11-Start-of-the-artMethodsComparison', criterion)
            # basic_file_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'Intra-Subject', '11-Start-of-the-artMethodsComparison', criterion)
            for subject_order in range(len(subjects_list_global)):
                subject = subjects_list_global[subject_order]
                metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
                # 判断文件是否存在
                if not os.path.exists(metrics_file_name):
                    print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                else:
                    # 读取每个受试者的test_metrics
                    print("读取受试者：%s 的test_metrics: " %subject)
                    df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                    # ignore_index=True参数用于重置索引，以确保索引是连续的
                    df1_metrics.extend(df.T.values[:-2, :])
                    df2_metrics_mean.append(df.T.values[-2, :])
                    df2_metrics_std.append(df.T.values[-1, :])
                
            printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
            df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
            mean_row.index = ['mean']  # 设置索引名称
            df1 = pd.concat([df1, mean_row])
            # 计算标准差并添加到DataFrame
            std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
            std_row.index = ['std']  # 设置索引名称
            df1 = pd.concat([df1, std_row]).round(3)
            # 保存df1  
            dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
            df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
            df1.to_csv(df1_save_name, index=True)
            
            printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
            # 保存df2
            df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
            df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
            df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
            df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
            # 计算平均值并添加到DataFrame
            mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
            std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
            df2.loc['mean'] = mean_row
            df2.loc['std'] = std_row
            df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
            df2.to_csv(df2_save_name, index=True)
            
            if gait_or_motion == 'gait' and train_test_utils['hmm_tools']['use_hmm']:
                ## test_metrics——hmm.csv处理
                # 新增一个记录所有受试者的所有测试结果的df1
                df1_metrics = []
                df2_metrics_mean = []
                df2_metrics_std = []
                for subject_order in range(len(subjects_list_global)):
                    subject = subjects_list_global[subject_order]
                    metrics_file_path = get_save_path(basic_file_path, gait_or_motion, motion_type, model_name, subject)
                    metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics_hmm.csv')
                    # 判断文件是否存在
                    if not os.path.exists(metrics_file_name):
                        print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
                    else:
                        # 读取每个受试者的test_metrics
                        print("读取受试者：%s 的test_metrics_hmm: " %subject)
                        df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                        # ignore_index=True参数用于重置索引，以确保索引是连续的
                        df1_metrics.extend(df.T.values[:-2, :])
                        df2_metrics_mean.append(df.T.values[-2, :])
                        df2_metrics_std.append(df.T.values[-1, :])
                
                printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
                df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
                mean_row.index = ['mean']  # 设置索引名称
                df1 = pd.concat([df1, mean_row])
                # 计算标准差并添加到DataFrame
                std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
                std_row.index = ['std']  # 设置索引名称
                df1 = pd.concat([df1, std_row]).round(3)
                # 保存df1  
                dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
                df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results_hmm.csv')
                df1.to_csv(df1_save_name, index=True)
                
                printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
                # 保存df2
                df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
                df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
                df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
                df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
                # 计算平均值并添加到DataFrame
                mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
                std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
                df2.loc['mean'] = mean_row
                df2.loc['std'] = std_row
                df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results_hmm.csv')
                df2.to_csv(df2_save_name, index=True)